In [2]:
import numpy as np 
import pandas as pd 
import xgboost as xgb
import random
import datetime as dt
import gc

import seaborn as sns 

np.random.seed(1)

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

zillow-prize-1



In [3]:
train_2016 = pd.read_csv('/kaggle/input/zillow-prize-1/train_2016_v2.csv' , parse_dates=["transactiondate"]) 
train_2017 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2017.csv') 
properties_2016 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2016.csv')   
properties_2017 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2017.csv')
test = pd.read_csv('/kaggle/input/zillow-prize-1/sample_submission.csv') 
test= test.rename(columns={'ParcelId': 'parcelid'})

/tmp/ipykernel_34/3062073402.py:2: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_2017 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2017.csv')
/tmp/ipykernel_34/3062073402.py:3: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  properties_2016 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2016.csv')
/tmp/ipykernel_34/3062073402.py:4: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  properties_2017 = pd.read_csv('/kaggle/input/zillow-prize-1/properties_2017.csv')


In [4]:
train = pd.concat([train_2016, train_2017])
properties = pd.concat([properties_2016, properties_2017])

In [6]:
print("Training Size:" + str(train.shape))
print("Property Size:" + str(properties.shape))
print("Sample Size:" + str(test.shape))

Training Size:(3075492, 60)
Property Size:(5970434, 58)
Sample Size:(2985217, 7)


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3075492 entries, 0 to 2985216
Data columns (total 60 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   parcelid                      int64         
 1   logerror                      float64       
 2   transactiondate               datetime64[ns]
 3   airconditioningtypeid         float64       
 4   architecturalstyletypeid      float64       
 5   basementsqft                  float64       
 6   bathroomcnt                   float64       
 7   bedroomcnt                    float64       
 8   buildingclasstypeid           float64       
 9   buildingqualitytypeid         float64       
 10  calculatedbathnbr             float64       
 11  decktypeid                    float64       
 12  finishedfloor1squarefeet      float64       
 13  calculatedfinishedsquarefeet  float64       
 14  finishedsquarefeet12          float64       
 15  finishedsquarefeet13          float64

In [8]:
train = train.drop('airconditioningtypeid', axis=1)
train = train.drop('architecturalstyletypeid', axis=1)
train = train.drop('basementsqft', axis=1)
train = train.drop('bathroomcnt', axis=1)
train = train.drop('bedroomcnt', axis=1)
train = train.drop('buildingclasstypeid', axis=1)
train = train.drop('buildingqualitytypeid', axis=1)
train = train.drop('calculatedbathnbr', axis=1)
train = train.drop('decktypeid', axis=1)
train = train.drop('finishedfloor1squarefeet', axis=1)
train = train.drop('calculatedfinishedsquarefeet', axis=1)
train = train.drop('finishedsquarefeet12', axis=1)
train = train.drop('finishedsquarefeet13', axis=1)
train = train.drop('finishedsquarefeet15', axis=1)
train = train.drop('finishedsquarefeet50', axis=1)
train = train.drop('finishedsquarefeet6', axis=1)
train = train.drop('fips', axis=1)
train = train.drop('fireplacecnt', axis=1)
train = train.drop('fullbathcnt', axis=1)
train = train.drop('garagecarcnt', axis=1)
train = train.drop('hashottuborspa', axis=1)
train = train.drop('garagetotalsqft', axis=1)
train = train.drop('heatingorsystemtypeid', axis=1)
train = train.drop('latitude', axis=1)
train = train.drop('longitude', axis=1)
train = train.drop('lotsizesquarefeet', axis=1)
train = train.drop('poolcnt', axis=1)
train = train.drop('poolsizesum', axis=1)
train = train.drop('pooltypeid10', axis=1)
train = train.drop('pooltypeid2', axis=1)
train = train.drop('pooltypeid7', axis=1)
train = train.drop('propertycountylandusecode', axis=1)
train = train.drop('propertylandusetypeid', axis=1)
train = train.drop('propertyzoningdesc', axis=1)
train = train.drop('rawcensustractandblock', axis=1)
train = train.drop('regionidcity', axis=1)
train = train.drop('regionidcounty', axis=1)
train = train.drop('regionidneighborhood', axis=1)
train = train.drop('regionidzip', axis=1)
train = train.drop('roomcnt', axis=1)
train = train.drop('storytypeid', axis=1)
train = train.drop('threequarterbathnbr', axis=1)
train = train.drop('typeconstructiontypeid', axis=1)
train = train.drop('unitcnt', axis=1)
train = train.drop('yardbuildingsqft17', axis=1)
train = train.drop('yardbuildingsqft26', axis=1)
train = train.drop('yearbuilt', axis=1)
train = train.drop('numberofstories', axis=1)
train = train.drop('fireplaceflag', axis=1)
train = train.drop('structuretaxvaluedollarcnt', axis=1)
train = train.drop('taxvaluedollarcnt', axis=1)
train = train.drop('assessmentyear', axis=1)
train = train.drop('landtaxvaluedollarcnt', axis=1)
train = train.drop('taxamount', axis=1)
train = train.drop('taxdelinquencyflag', axis=1)
train = train.drop('taxdelinquencyyear', axis=1)
train = train.drop('censustractandblock', axis=1)


In [9]:
train['transactiondate'] = train['transactiondate'].astype('int64')

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3075492 entries, 0 to 2985216
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   parcelid         int64  
 1   logerror         float64
 2   transactiondate  int64  
dtypes: float64(1), int64(2)
memory usage: 93.9 MB


In [11]:
print("Sample Size:" + str(test.shape))

Sample Size:(2985217, 7)


In [12]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,1451606400000000000
1,14366692,-0.1684,1451606400000000000
2,12098116,-0.0040,1451606400000000000
3,12643413,0.0218,1451692800000000000
4,14432541,-0.0050,1451692800000000000


In [13]:
train_dummies = pd.get_dummies(train)
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,1451606400000000000
1,14366692,-0.1684,1451606400000000000
2,12098116,-0.0040,1451606400000000000
3,12643413,0.0218,1451692800000000000
4,14432541,-0.0050,1451692800000000000


In [14]:
for c, dtype in zip(properties.columns, properties.dtypes):
    if dtype == np.float64:        
        properties[c] = properties[c].astype(np.float32)
    if dtype == np.int64:
        properties[c] = properties[c].astype(np.int32)

for column in test.columns:
    if test[column].dtype == int:
        test[column] = test[column].astype(np.int32)
    if test[column].dtype == float:
        test[column] = test[column].astype(np.float32)

In [15]:
print("Sample Size:" + str(test.shape))

Sample Size:(2985217, 7)


In [16]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5970434 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      int32  
 1   airconditioningtypeid         float32
 2   architecturalstyletypeid      float32
 3   basementsqft                  float32
 4   bathroomcnt                   float32
 5   bedroomcnt                    float32
 6   buildingclasstypeid           float32
 7   buildingqualitytypeid         float32
 8   calculatedbathnbr             float32
 9   decktypeid                    float32
 10  finishedfloor1squarefeet      float32
 11  calculatedfinishedsquarefeet  float32
 12  finishedsquarefeet12          float32
 13  finishedsquarefeet13          float32
 14  finishedsquarefeet15          float32
 15  finishedsquarefeet50          float32
 16  finishedsquarefeet6           float32
 17  fips                          float32
 18  fireplacecnt               

In [17]:
properties['living_area_prop'] = properties['calculatedfinishedsquarefeet'] / properties['lotsizesquarefeet']
properties['value_ratio'] = properties['taxvaluedollarcnt'] / properties['taxamount']
properties['value_prop'] = properties['structuretaxvaluedollarcnt'] / properties['landtaxvaluedollarcnt']


df_train = train.merge(properties, how='left', on='parcelid') 
df_test = test.merge(properties, how='left', on='parcelid')


del properties, train
gc.collect();

print('Memory usage reduction...')
df_train[['latitude', 'longitude']] /= 1e6
df_test[['latitude', 'longitude']] /= 1e6

df_train['censustractandblock'] /= 1e12
df_test['censustractandblock'] /= 1e12

print(df_train.fireplaceflag.isnull().sum())
print(df_train.fireplacecnt.isnull().sum())

df_train['fireplaceflag']= "No"
df_train.loc[df_train['fireplacecnt']>0,'fireplaceflag']= "Yes"

index = df_train.fireplacecnt.isnull()
df_train.loc[index,'fireplacecnt'] = 0

index = df_train.taxdelinquencyflag.isnull()
df_train.loc[index,'taxdelinquencyflag'] = "None"


print(df_train.hashottuborspa.value_counts())
print(df_train.pooltypeid10.value_counts())

print(df_train.hashottuborspa.value_counts())
print(df_train.pooltypeid10.value_counts())

index = df_train.pooltypeid2.isnull()
df_train.loc[index,'pooltypeid2'] = 0

index = df_train.pooltypeid7.isnull()
df_train.loc[index,'pooltypeid7'] = 0

index = df_train.poolcnt.isnull()
df_train.loc[index,'poolcnt'] = 0

Memory usage reduction...
6140214
5505974
hashottuborspa
True    123219
Name: count, dtype: int64
pooltypeid10
1.0    55666
Name: count, dtype: int64
hashottuborspa
True    123219
Name: count, dtype: int64
pooltypeid10
1.0    55666
Name: count, dtype: int64


In [18]:
print("Sample Size:" + str(test.shape))

Sample Size:(2985217, 7)


In [19]:
from sklearn.preprocessing import LabelEncoder  

lbl = LabelEncoder()
for c in df_train.columns:
    df_train[c]=df_train[c].fillna(0)
    if df_train[c].dtype == 'object':
        lbl.fit(list(df_train[c].values))
        df_train[c] = lbl.transform(list(df_train[c].values))

for c in df_test.columns:
    df_test[c]=df_test[c].fillna(0)
    if df_test[c].dtype == 'object':
        lbl.fit(list(df_test[c].values))
        df_test[c] = lbl.transform(list(df_test[c].values))   

In [20]:
print("Sample Size:" + str(test.shape))

Sample Size:(2985217, 7)


In [21]:
log_errors = df_train['logerror']
df_train = df_train[df_train.logerror < np.percentile(log_errors, 99.5)]
df_train = df_train[df_train.logerror > np.percentile(log_errors, 0.5)]

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 
                         'propertycountylandusecode' ], axis=1)

x_test = df_test.drop(['parcelid', 'propertyzoningdesc',
                       'propertycountylandusecode', '201610', '201611', 
                       '201612', '201710', '201711', '201712'], axis = 1) 

x_train = x_train.values
y_train = df_train['logerror'].values

from sklearn.model_selection import train_test_split

X = x_train
y = y_train 

Xtrain, Xvalid, ytrain, yvalid = train_test_split(X, y, test_size=0.2, random_state=42)

dtrain = xgb.DMatrix(Xtrain, label=ytrain)
dvalid = xgb.DMatrix(Xvalid, label=yvalid)
dtest = xgb.DMatrix(x_test.values)
 
xgb_params = {'min_child_weight': 5, 'eta': 0.035, 'colsample_bytree': 0.5, 'max_depth': 4,
            'subsample': 0.85, 'lambda': 0.8, 'nthread': -1, 'booster' : 'gbtree', 'silent': 1, 'gamma' : 0,
            'eval_metric': 'mae', 'objective': 'reg:linear' }           

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

model_xgb = xgb.train(xgb_params, dtrain, 1000, watchlist, early_stopping_rounds=100,
                  maximize=False, verbose_eval=10)

Predicted_test_xgb = model_xgb.predict(dtest)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:00:15] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:00:15] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mae:0.00053	valid-mae:0.00052
[10]	train-mae:0.00053	valid-mae:0.00052
[20]	train-mae:0.00053	valid-mae:0.00052
[30]	train-mae:0.00053	valid-mae:0.00052
[40]	train-mae:0.00053	valid-mae:0.00052
[50]	train-mae:0.00053	valid-mae:0.00052
[60]	train-mae:0.00053	valid-mae:0.00052
[70]	train-mae:0.00053	valid-mae:0.00052
[80]	train-mae:0.00053	valid-mae:0.00052
[90]	train-mae:0.00053	valid-mae:0.00052
[100]	train-mae:0.00053	valid-mae:0.00052
[110]	train-mae:0.00053	valid-mae:0.00052
[120]	train-mae:0.00053	valid-mae:0.00052
[130]	train-mae:0.00053	valid-mae:0.00052
[140]	train-mae:0.00053	valid-mae:0.00052
[150]	train-mae:0.00053	valid-mae:0.00052
[160]	train-mae:0.00053	valid-mae:0.00052
[170]	train-mae:0.00053	valid-mae:0.00052
[180]	train-mae:0.00053	valid-mae:0.00052
[190]	train-mae:0.00053	valid-mae:0.00052
[200]	train-mae:0.00053	valid-mae:0.00052
[210]	train-mae:0.00053	valid-mae:0.00052
[220]	train-mae:0.00053	valid-mae:0.00052
[230]	train-mae:0.00053	valid-mae:0.00052
[23

In [24]:
print("Sample Size:" + str(test.shape))
print("Sample Size:" + str(Predicted_test_xgb.shape))

Sample Size:(2985217, 7)
Sample Size:(5970434,)


In [23]:
sample_file = pd.read_csv('/kaggle/input/zillow-prize-1/sample_submission.csv') 
for c in sample_file.columns[sample_file.columns != 'ParcelId']:
    sample_file[c] = Predicted_test_xgb

print('Preparing the csv file ...')
sample_file.to_csv('xgb_predicted_results.csv', index=False, float_format='%.4f')
print("Finished writing the file")

ValueError: Length of values (5970434) does not match length of index (2985217)

In [99]:
sample_file = pd.read_csv('/kaggle/input/zillow-prize-1/sample_submission.csv')

In [101]:
sample_file


,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0
2985213,14273630,0,0,0,0,0,0
2985214,168040630,0,0,0,0,0,0
2985215,168040830,0,0,0,0,0,0


In [102]:
df_test['parcelid']

0           10754147
1           10754147
2           10759547
3           10759547
4           10843547
             ...    
5970429    168040630
5970430    168040830
5970431    168040830
5970432    168040430
5970433    168040430
Name: parcelid, Length: 5970434, dtype: int32